In [1]:
# Connect the colab notebook with drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import metrics

In [3]:
dataset =  pd.read_csv("/content/drive/MyDrive/Customer_Churn/Cleaned_data_24_10_22.csv")
dataset.head(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df = dataset[['Geography', 'Gender','Age','Tenure','Balance','NumOfProducts','HasCrCard','IsActiveMember','Exited']].copy()
df.head(5)

,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,0,0,42,2,0.00,1,1,1,1
1,2,0,41,1,83807.86,1,0,1,0
2,0,0,42,8,159660.80,3,1,0,1
3,0,0,39,1,0.00,2,0,0,0
4,2,0,43,2,125510.82,1,1,1,0


In [5]:
df.shape

(15682, 9)

In [6]:
X = dataset.drop(['Exited'],axis=1)
y = dataset['Exited']

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

In [9]:
# import the libraries
import tensorflow as tf
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


In [10]:
def build_clf(unit):
# creating the layers of the NN
  ann = tf.keras.models.Sequential()
  ann.add(tf.keras.layers.Dense(units=unit, activation='relu'))
  ann.add(tf.keras.layers.Dense(units=unit, activation='relu'))
  ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
  ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return ann


In [11]:
model=KerasClassifier(build_fn=build_clf)


<ipython-input-11-94990c6d4461>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=build_clf)


In [12]:
params={'batch_size':[100, 20, 50, 25, 32],
		'nb_epoch':[200, 100, 300, 400],
		'unit':[5,6, 10, 11, 12, 15],
		
		}
model=RandomizedSearchCV(estimator=model, param_distributions=params, cv=10)
# now fit the dataset to the GridSearchCV object.
result = model.fit(X_train, y_train)


628/628 [==============================] - 1s 1ms/step - loss: 0.5014 - accuracy: 0.7618


In [13]:
# Predict values using the training data
ann_cv_predict_train = model.predict(X_train)

print("Accuracy against training data: {0:.4f}".format(metrics.accuracy_score(y_train, ann_cv_predict_train)))
print()

393/393 [==============================] - 0s 867us/step
Accuracy against training data: 0.7971



In [14]:
# Predict values using the test data
ann_cv_predict_test = model.predict(X_test)

print("Accuracy against test data: {0:.4f}".format(metrics.accuracy_score(y_test, ann_cv_predict_test)))
print()

99/99 [==============================] - 0s 881us/step
Accuracy against test data: 0.7928



In [15]:
print("Confusion Matrix")
print(metrics.confusion_matrix(y_test, ann_cv_predict_test))
print()

Confusion Matrix
[[1250  311]
 [ 339 1237]]



In [16]:
print("Classification Report")
print(metrics.classification_report(y_test, ann_cv_predict_test))
print()

Classification Report
              precision    recall  f1-score   support

           0       0.79      0.80      0.79      1561
           1       0.80      0.78      0.79      1576

    accuracy                           0.79      3137
   macro avg       0.79      0.79      0.79      3137
weighted avg       0.79      0.79      0.79      3137


